In [1]:
import numpy as np
import cv2
import keras

#haar_path = 'C:/Users/Ray/Anaconda3/envs/tensorflow_env2/Library/etc/haarcascades/'

#face_cascade = cv2.CascadeClassifier(haar_path+'haarcascade_frontalface_default.xml')

face_cascade = cv2.CascadeClassifier('C:/Users/Ray/Anaconda3/pkgs/opencv-4.0.1-py37hce2de41_201/Library/etc/haarcascades/haarcascade_frontalface_default.xml')
# eye_cascade = cv.CascadeClassifier(haar_path+'haarcascade_eye.xml')

from keras.preprocessing.image import img_to_array
from keras.models import load_model
import imutils
import cv2
import numpy as np
import sys

# parameters for loading data and images
#detection_model_path = haar_path+'haarcascade_frontalface_default.xml'
emotion_model_path = 'C:/Users/Ray/Desktop/HKUST/CSIT6000F/project/Emotion_Detection/models/model_Full_Train.hdf5'

# hyper-parameters for bounding boxes shape
# loading models
#face_detection = cv2.CascadeClassifier(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)
EMOTIONS = ["angry", "disgust", "scared", "happy", "sad", "surprised", "neutral"]
canvas_width = 600
canvas_height = 500
bar_width = int((canvas_width - 20) / (len(EMOTIONS)))
bar_sep_width = 30
fontscale = 0.5
color = (240, 176, 0)

# starting video streaming
cv2.namedWindow('your_face')
camera = cv2.VideoCapture(0)

while True:

    frame = camera.read()[1]
    # reading the frame
    frame = imutils.resize(frame, width=400)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30),
                                            flags=cv2.CASCADE_SCALE_IMAGE)
    canvas = np.zeros((canvas_height, canvas_width, 3), dtype="uint8")
    frameClone = frame.copy()

    if len(faces) > 0:
        faces = sorted(faces, reverse=True,
                       key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = faces

        # Extract the ROI of the face from the grayscale image, resize it to a fixed 48x48 pixels, and then prepare
        # the ROI for classification via the CNN
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)

        preds = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]

    for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
        # construct the label text
        text_emo = "{}".format(emotion)
        text_prob = "{:.2f}%".format(prob * 100)
        w = int(prob * canvas_width)

        # Probability Screen
        cv2.rectangle(canvas, ((i * bar_width) + bar_sep_width, canvas_height-10),
                      ((i * bar_width) + bar_width,canvas_height-w), color, -1)
        cv2.putText(canvas, text_emo, ((i * bar_width) + 30 ,canvas_height - 50),
                    cv2.FONT_HERSHEY_SIMPLEX, fontscale,
                    (255, 255, 255), 2)
        cv2.putText(canvas, text_prob, ((i * bar_width) + 30, canvas_height - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, fontscale,
                    (255, 255, 255), 2)

        # Face Screen
        cv2.putText(frameClone, label, (fX, fY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, fontscale, color, 2)
        cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),
                      color, 5)

    cv2.imshow('your_face', frameClone)
    cv2.imshow("Probabilities", canvas)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()


Using TensorFlow backend.


KeyboardInterrupt: 